## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.tree2 = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.max_features = max_features
        self.num_class = -1
        self.tree_level = 0
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print ('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s,all_count):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        N_l = 1 - pow(l_c/l_s,2).sum(axis=1)
        N_r = 1 - pow(r_c/r_s,2).sum(axis=1)
        N = sum(all_count) 
        r = np.transpose((r_s/N).reshape(len(N_r)))*N_r
        l = np.transpose((l_s/N).reshape(len(N_l)))*N_l
        N_all = 1 - sum(pow(all_count/N,2))
        nGain = N_all -  l - r
        nGain =  np.around(nGain, decimals=3)   
        #print('--gini--')
        #print(N_all,r,l,nGain)
  
        return nGain
    
    def __entropy(self, l_c, l_s, r_c, r_s,all_count):
        
        N_l = - sum(l_c/l_s*np.log2(l_c/l_s)) 
        N_r = - sum(r_c/r_s*np.log2(r_c/r_s)) 
        N = sum(all_count) 
        nGain = -  l - r
        nGain =  np.around(nGain, decimals=3)   
        #print('-----')
        #print(me_all_mx, r, l,nGain)
        return nGain

    def __misclass(self, l_c, l_s, r_c, r_s, all_count):
        
        me_l = 1 - np.amax(l_c/l_s, axis=1) 
        me_r = 1 - np.amax(r_c/r_s, axis=1) 
        N = sum(all_count) 
        me_all = 1 - max(all_count/N)
        ones = np.ones(me_r.shape[0]) #нулевая матрица размерность класса
        me_all_mx = ones*me_all
        r = np.transpose((r_s/N).reshape(len(me_r)))*me_r
        l = np.transpose((l_s/N).reshape(len(me_l)))*me_l
        nGain = me_all_mx -  l - r
        nGain =  np.around(nGain, decimals=3)   
        
        
        return nGain

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange((n_feature))
        fts = shuffle(feature_ids)
        num = np.sqrt(n_feature)
        return fts[0:int(num)]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange((n_feature))
        fts = shuffle(feature_ids)
        num = np.log(n_feature)
        return fts[0:int(num)]

    def __get_feature_ids_N(self, n_feature):
        feature_ids = np.arange((n_feature))
        fts= shuffle(feature_ids)
        num = len(fts)
        return fts[0:int(num)]
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        #left_mask = x > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __find_depth(self, node_id):    
        if node_id < 0:
            return -1
        if node_id == 0:
            return 0
        node = self.tree2[node_id]
        parentId = node['parentId']
        depth = 1
        while parentId != 0:
            node =  self.tree2[parentId]
            parentId = node['parentId']
            depth +=1
        return depth

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        #  Сортирует исходные данные по количественному признаку, для нахождения порогов
        #  находит количество уникальных классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        num_class = np.unique(y).size
   
        # Что делает этот блок кода?
        # выбирает min_samples_split элементов с обоих концов, те центральных min_samples_split
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        #находит индексы, на которых происходит смена класса
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
     
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        #Строит матрицу возможных разбиений узла, каждая срока представляет собой количество элементов, каждого класса
        # матрица нужна для расчета информативности каждого разбиения
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], num_class)) #нулевая матрица размерность класса
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=num_class)
     
        # Что делает этот блок кода?
        # строит матрицы размерностью n на m, где m - количество уникальных классов.Каждая строка содержит
        # количество представительей каждого класса, при каждом разбиении
        all_class_count = np.bincount(y)
        l_class_count = np.cumsum(class_increments, axis=0)  
        r_class_count = all_class_count - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        
       
        #Расчет информативности 
        # Что делает этот блок кода?
        # определяет матрицу прироста информации для каждого разбиения
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes,all_class_count )
        idx = np.argmin(gs)
        #print('-gs, idx-')
        #print(gs, idx)
        
        
        # Что делает этот блок кода?
        # Возвращает выбраннную информативность
        # и Определяет порог(условие), по которому будет происходить разбиение данных  
        left_el_id = l_sizes[idx][0]
        
        
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1, prev='', parentId =-1):
        #print('----------FIT_NODE-------')
        #print(len(x))
        if len(x) <= 5:
            self.tree2[node_id] = {'type':self.__class__.LEAF_TYPE} 
            prev ='right'
        
        else:
            self.nums = len(x)
            # Ваш код
           # self.sufficient_share
            
            self.max_features = self.max_features or x.shape[1]
            feature_ids = self.get_feature_ids(self.max_features); 
            threshold_total = dict()
            for f_id in feature_ids:
                x_cl = x[:,f_id]
                threshold_total[f_id] = self.__find_threshold(x_cl,y)
            #print(threshold_total)    
            feature_idx = min(threshold_total, key=threshold_total.get)
            threshold = threshold_total[feature_idx]
            if threshold[1] == None: 
                self.tree[node_id] = {'level':0,'type':self.__class__.LEAF_TYPE, 'feature_id':feature_idx, 'threshold':threshold[1], 'left_x': x, 'right_x': {}, 'left_y': y, 'right_y': {}} 
                self.tree2[node_id] = {'level':0,'type':self.__class__.LEAF_TYPE, 'feature_id':feature_idx, 'threshold':threshold[1], 'left_x': len(x), 'right_x': 0, 'parentId':parentId} 
                prev ='right'
            else:
                node = self.__div_samples(x,y,feature_idx,threshold[1])
                self.tree[node_id] = {'level':0, 'type':self.__class__.NON_LEAF_TYPE, 'feature_id':feature_idx, 'threshold':threshold[1], 'left_x': node[0], 'right_x': node[1], 'left_y': node[2], 'right_y': node[3]} 
                self.tree2[node_id] = {'level':0, 'type':self.__class__.NON_LEAF_TYPE, 'feature_id':feature_idx, 'threshold':threshold[1], 'left_x': len(node[0]), 'right_x': len(node[1]),'parentId':parentId} 
            #нужно ли запускать дальше 
            depth = self.__find_depth(node_id)
            if self.max_depth != None and depth >= self.max_depth:
                return 0
            node_id += 1
            if prev == '' or prev == 'right':
                prev = 'left'
                parentId = parentId + 1
                self.__fit_node(self.tree[parentId]['left_x'],self.tree[parentId]['left_y'], node_id,0,pred_f,'left',parentId)

            elif prev == 'left' :
                prev= 'right'
                self.__fit_node(self.tree[parentId]['right_x'],self.tree[parentId]['right_y'], node_id,0,pred_f,'right',parentId)
        return 1
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 
        print(self.tree2)
        return self.tree2
        
    def __predict_class(self, x, node_id):
        node = self.tree2[node_id]
        if node['type'] == self.__class__.NON_LEAF_TYPE and (2 * node_id + 1) <= max(self.tree):
            feature_id = node['feature_id']
            threshold = node['threshold']
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return self.tree2[node_id]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data[:90]
Y = iris.target[:90]
X_test = iris.data[97:99]
Y_test = iris.target[97:99]
X_test, Y_test

(array([[6.2, 2.9, 4.3, 1.3],
        [5.1, 2.5, 3. , 1.1]]), array([1, 1]))

In [4]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2,criterion='gini', max_depth = 4)
tree = my_clf.fit(X,Y)

{0: {'level': 0, 'type': 0, 'feature_id': 1, 'threshold': 2.3499999999999996, 'left_x': 84, 'right_x': 6, 'parentId': -1}, 1: {'level': 0, 'type': 0, 'feature_id': 0, 'threshold': 4.9, 'left_x': 64, 'right_x': 20, 'parentId': 0}, 2: {'level': 0, 'type': 0, 'feature_id': 1, 'threshold': 2.25, 'left_x': 3, 'right_x': 3, 'parentId': 0}, 3: {'level': 0, 'type': 0, 'feature_id': 0, 'threshold': 5.65, 'left_x': 28, 'right_x': 36, 'parentId': 1}, 4: {'level': 0, 'type': 1, 'feature_id': 1, 'threshold': None, 'left_x': 20, 'right_x': 0, 'parentId': 1}, 5: {'type': 1}}


In [5]:
d = my_clf.predict(X_test)
d

array([{'level': 0, 'type': 0, 'feature_id': 0, 'threshold': 5.65, 'left_x': 28, 'right_x': 36, 'parentId': 1},
       {'level': 0, 'type': 0, 'feature_id': 0, 'threshold': 5.65, 'left_x': 28, 'right_x': 36, 'parentId': 1}],
      dtype=object)

In [6]:
df = pd.read_csv('e:\DS\ds_hw\cs_training.csv', sep=',').dropna()
df.shape

(120269, 11)

In [7]:
X = df.as_matrix(columns=df.columns[1:])
Y = df.as_matrix(columns=df.columns[:1])
Y = Y.reshape(Y.shape[0])


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [15]:
my_clf.predict(X_test)

array([{'level': 0, 'type': 0, 'feature_id': 8, 'threshold': 0.0, 'left_x': 1971, 'right_x': 71106, 'parentId': 7},
       {'level': 0, 'type': 0, 'feature_id': 8, 'threshold': 0.0, 'left_x': 1971, 'right_x': 71106, 'parentId': 7},
       {'level': 0, 'type': 0, 'feature_id': 8, 'threshold': 0.0, 'left_x': 1971, 'right_x': 71106, 'parentId': 7},
       ...,
       {'level': 0, 'type': 0, 'feature_id': 8, 'threshold': 0.0, 'left_x': 1971, 'right_x': 71106, 'parentId': 7},
       {'level': 0, 'type': 0, 'feature_id': 5, 'threshold': 0.0, 'left_x': 30, 'right_x': 19, 'parentId': 10},
       {'level': 0, 'type': 0, 'feature_id': 8, 'threshold': 0.0, 'left_x': 1971, 'right_x': 71106, 'parentId': 7}],
      dtype=object)

In [8]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2,criterion='gini')
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [10]:
t1 = time()
my_clf.fit(X, Y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(X, Y)
t2 = time()
print(t2 - t1)

{0: {'level': 0, 'type': 0, 'feature_id': 2, 'threshold': 0.0, 'left_x': 20299, 'right_x': 99970, 'parentId': -1}, 1: {'level': 0, 'type': 0, 'feature_id': 4, 'threshold': 0.0, 'left_x': 20103, 'right_x': 196, 'parentId': 0}, 2: {'level': 0, 'type': 0, 'feature_id': 2, 'threshold': 0.0, 'left_x': 0, 'right_x': 99970, 'parentId': 0}, 3: {'level': 0, 'type': 0, 'feature_id': 8, 'threshold': 0.0, 'left_x': 3600, 'right_x': 16503, 'parentId': 1}, 4: {'level': 0, 'type': 0, 'feature_id': 7, 'threshold': 0.0, 'left_x': 100, 'right_x': 96, 'parentId': 1}, 5: {'type': 1}, 6: {'level': 0, 'type': 0, 'feature_id': 8, 'threshold': 0.0, 'left_x': 18, 'right_x': 1079, 'parentId': 2}, 7: {'level': 0, 'type': 0, 'feature_id': 4, 'threshold': 0.0, 'left_x': 5828, 'right_x': 46, 'parentId': 3}, 8: {'level': 0, 'type': 0, 'feature_id': 6, 'threshold': 0.0, 'left_x': 4108, 'right_x': 101022, 'parentId': 3}, 9: {'level': 0, 'type': 0, 'feature_id': 2, 'threshold': 0.0, 'left_x': 63, 'right_x': 367, 'paren

## Проверка качества работы

In [11]:
gkf = KFold(n_splits=5, shuffle=True)

In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

# Применить для задачи Titanic 

In [44]:
df = pd.read_csv("e:/DS/ds_hw/titanic/train.csv", sep=',').dropna()


In [37]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [45]:
df['iSex'] = np.where(df['Sex']=='male', 1, 0)


In [46]:
cols = ['Survived', 'Pclass', 'iSex', 'Age', 'SibSp', 'Parch', 'Fare']
df = df[cols]

In [47]:
X = df.as_matrix(columns=df.columns[1:])
Y = df.as_matrix(columns=df.columns[:1])
Y = Y.reshape(Y.shape[0])

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [49]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2,criterion='gini', max_depth = 4)

In [50]:
my_clf.fit(X,Y)

{0: {'level': 0, 'type': 0, 'feature_id': 3, 'threshold': 0.0, 'left_x': 73, 'right_x': 110, 'parentId': -1}, 1: {'level': 0, 'type': 0, 'feature_id': 4, 'threshold': 0.0, 'left_x': 30, 'right_x': 43, 'parentId': 0}, 2: {'level': 0, 'type': 0, 'feature_id': 4, 'threshold': 0.0, 'left_x': 31, 'right_x': 79, 'parentId': 0}, 3: {'level': 0, 'type': 0, 'feature_id': 4, 'threshold': 1.0, 'left_x': 13, 'right_x': 17, 'parentId': 1}, 4: {'level': 0, 'type': 0, 'feature_id': 4, 'threshold': 0.0, 'left_x': 0, 'right_x': 43, 'parentId': 1}, 5: {'level': 0, 'type': 0, 'feature_id': 1, 'threshold': 1.0, 'left_x': 0, 'right_x': 31, 'parentId': 2}, 6: {'level': 0, 'type': 0, 'feature_id': 4, 'threshold': 0.0, 'left_x': 0, 'right_x': 79, 'parentId': 2}, 7: {'level': 0, 'type': 0, 'feature_id': 3, 'threshold': 2.5, 'left_x': 3, 'right_x': 10, 'parentId': 3}, 8: {'level': 0, 'type': 0, 'feature_id': 0, 'threshold': 1.0, 'left_x': 6, 'right_x': 11, 'parentId': 3}, 9: {'type': 1}}


{0: {'level': 0,
  'type': 0,
  'feature_id': 3,
  'threshold': 0.0,
  'left_x': 73,
  'right_x': 110,
  'parentId': -1},
 1: {'level': 0,
  'type': 0,
  'feature_id': 4,
  'threshold': 0.0,
  'left_x': 30,
  'right_x': 43,
  'parentId': 0},
 2: {'level': 0,
  'type': 0,
  'feature_id': 4,
  'threshold': 0.0,
  'left_x': 31,
  'right_x': 79,
  'parentId': 0},
 3: {'level': 0,
  'type': 0,
  'feature_id': 4,
  'threshold': 1.0,
  'left_x': 13,
  'right_x': 17,
  'parentId': 1},
 4: {'level': 0,
  'type': 0,
  'feature_id': 4,
  'threshold': 0.0,
  'left_x': 0,
  'right_x': 43,
  'parentId': 1},
 5: {'level': 0,
  'type': 0,
  'feature_id': 1,
  'threshold': 1.0,
  'left_x': 0,
  'right_x': 31,
  'parentId': 2},
 6: {'level': 0,
  'type': 0,
  'feature_id': 4,
  'threshold': 0.0,
  'left_x': 0,
  'right_x': 79,
  'parentId': 2},
 7: {'level': 0,
  'type': 0,
  'feature_id': 3,
  'threshold': 2.5,
  'left_x': 3,
  'right_x': 10,
  'parentId': 3},
 8: {'level': 0,
  'type': 0,
  'feature_i